In [1]:
import psycopg2
import random
import pandas as pd
import requests

In [2]:
streets_df = pd.read_csv('streetnames-southtyrol_20170608.csv', delimiter=';')

In [3]:
res = requests.get('http://uinames.com/api/?region=austria&amount=5')
managers = res.json()

In [4]:
conn = psycopg2.connect(dbname='admt_g02', user='admt_g02', password='xi6aeSoo', host='alcor.inf.unibz.it', port=5433)
cur = conn.cursor()

In [5]:
cur.execute('Delete from warehouse.showroom')

cur.execute('Select distinct district FROM warehouse.location where province = \'Bozen\'')
districts = cur.fetchall()

In [6]:
for district in districts:
    cur.execute('Select city FROM warehouse.location where district = \'%s\'' % district[0])
    cities = cur.fetchall()
    if(len(cities ) > 0):
        city = cities[random.randrange(len(cities))][0]
        
        streets = streets_df.loc[streets_df['GEMEINDE_COMUNE_DE'].str.slice(0,4).str.lower()== city.lower()[:4]].sample(1)
        street = streets['STRASSE_VIA_DE'].iloc[0]
        address = f'{street} - {random.randint(1, 100)}'
        
        manager = managers[random.randrange(len(managers))]
        
        telephone = f'047{random.randint(0,9)} - {"".join([str(random.randint(0,9)) for _ in range(6)])}' 
    
        cur.execute(f'Insert into warehouse.showroom (showroom_name, showroom_telephone, showroom_address, \
        showroom_size, showroom_manager, location_id) Values (\'Showroom-{city}\', \'{telephone}\', \'{address}\', \
        {random.randint(100, 1000)}, \'{manager["name"] + " " + manager["surname"]}\', \
        (Select location_id from warehouse.location where city = \'{city}\'))')

In [7]:
conn.commit()
cur.close()
conn.close()